In [2]:
import ee
import geemap
import pandas as pd
from datetime import datetime
import os

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project = "ee-samettenborough")

out_dir = os.path.expanduser("~\Downloads")

out_dir

<>:11: SyntaxWarning: invalid escape sequence '\D'
<>:11: SyntaxWarning: invalid escape sequence '\D'


C:\Users\samet\AppData\Local\Temp\ipykernel_38820\143372900.py:11: SyntaxWarning: invalid escape sequence '\D'
  out_dir = os.path.expanduser("~\Downloads")


'C:\\Users\\samet\\Downloads'

In [135]:


Map = geemap.Map()

image = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")

landsat_vis = {"bands": ["B4", "B3", "B2"], "gamma": 2}
Map.addLayer(image, landsat_vis, "LE7_TOA_5YEAR/1999_2003", True, 1)
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [141]:


region = ee.Geometry.BBox(-115.6339, 35.7529, -114.9338, 36.0)
Map.addLayer(region, {}, "region")
Map.centerObject(region)


In [142]:


features = geemap.fishnet(region, rows=2, cols=2)
Map.addLayer(features, {}, "Grids")


In [126]:
Map = geemap.Map()

image = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")

landsat_vis = {"bands": ["B4", "B3", "B2"], "gamma": 2}
Map.addLayer(image, landsat_vis, "LE7_TOA_5YEAR/1999_2003", True, 1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [127]:
region = ee.Geometry.BBox(-90.1517107831, 15.5726233568, -89.1726311637, 16.29215598)
Map.centerObject(region)

In [133]:
features = geemap.fishnet(region, rows = 2, cols = 2)
Map.addLayer(features, {}, "Grids")

In [134]:
geemap.download_ee_image_tiles(
    image, features, out_dir, prefix="landsat_", crs="EPSG:3857", scale=100
)

landsat_1.tif: |                                                      | 0.00/1.59M (raw) [  0.0%] in 00:00 (et…

landsat_2.tif: |                                                      | 0.00/1.59M (raw) [  0.0%] in 00:00 (et…

landsat_3.tif: |                                                      | 0.00/1.59M (raw) [  0.0%] in 00:00 (et…

landsat_4.tif: |                                                      | 0.00/1.60M (raw) [  0.0%] in 00:00 (et…

landsat_5.tif: |                                                      | 0.00/1.60M (raw) [  0.0%] in 00:00 (et…

landsat_6.tif: |                                                      | 0.00/1.60M (raw) [  0.0%] in 00:00 (et…

landsat_7.tif: |                                                      | 0.00/1.60M (raw) [  0.0%] in 00:00 (et…

landsat_8.tif: |                                                      | 0.00/1.60M (raw) [  0.0%] in 00:00 (et…

landsat_9.tif: |                                                      | 0.00/1.60M (raw) [  0.0%] in 00:00 (et…

Downloaded 9 tiles in 23.99776554107666 seconds.


In [9]:
out_dir = os.path.expanduser("~/Downloads")

In [14]:


loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(loc)
    .filterDate("2008-01-01", "2020-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)
Map = geemap.Map()
landsat_vis = {"bands": ["R", "G", "B"], "gamma": 2}
Map.addLayer(collection, landsat_vis, "testing", True, 1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
geemap.download_ee_image_collection(collection, out_dir, scale=10)

Total number of images: 9



m_4609915_sw_14_060_20180902_20181213.tif: |                          | 0.00/1.49M (raw) [  0.0%] in 00:00 (et…

m_4609915_sw_14_060_20190626.tif: |                                   | 0.00/1.54M (raw) [  0.0%] in 00:00 (et…

m_4609915_sw_14_1_20090818.tif: |                                     | 0.00/1.72M (raw) [  0.0%] in 00:00 (et…

m_4609915_sw_14_1_20100629.tif: |                                     | 0.00/1.66M (raw) [  0.0%] in 00:00 (et…

m_4609915_sw_14_1_20120714.tif: |                                     | 0.00/1.66M (raw) [  0.0%] in 00:00 (et…

m_4609915_sw_14_1_20140901.tif: |                                     | 0.00/1.66M (raw) [  0.0%] in 00:00 (et…

m_4609915_sw_14_1_20150926.tif: |                                     | 0.00/1.66M (raw) [  0.0%] in 00:00 (et…

m_4609915_sw_14_h_20160704.tif: |                                     | 0.00/1.64M (raw) [  0.0%] in 00:00 (et…

m_4609915_sw_14_h_20170703.tif: |                                     | 0.00/1.49M (raw) [  0.0%] in 00:00 (et…

## The section below works!

This code defines an aoi (region), the gets the collection, filters between specific dates, filters to the region, then removes any pixels where the cloud cover is more than 5. After that, we take the median value of remaining pixels to get a composite image. That composite image now represents the "cloudless" image for the region. If we keep the time window pretty small (1-2 months) we should be ok.

There are some commented out sections that download the data but the data for this region is large. The fishnet and tile function should probably be used to split the images up but it adds an additional row and col of data outside of the region defined. Will need to make the region smaller to account for this. The other option just downloads the the fewest number of full scenes to include the full region. This can also be quite large and each image will be many gigs potentially.

In [3]:
region = ee.Geometry.BBox(-89.8867, 15.4242, -89.2083, 15.924)

# Load the LANDSAT/LC08/C02/T1_L2 collection and filter it
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2024-01-01', '2024-03-01') \
    .filterBounds(region) \
    .filter(ee.Filter.lte('CLOUD_COVER', 5))

# Create a cloud-free composite using the median reducer
composite = collection.median()

# # Define visualization parameters
# vis_params = {
#     'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # Red, Green, Blue
#     'min': 0,
#     'max': 20000,
# }

# # Create a map centered on Rhode Island
# Map = geemap.Map(center=[15.5, -90.15], zoom=10)
# Map.addLayer(composite.clip(region), vis_params, 'Rhode Island Composite')

# # Display the map
# Map

In [144]:
# features = geemap.fishnet(region, rows=2, cols = 2)
# Map.addLayer(features, {}, "Grids")

In [4]:
# geemap.download_ee_image_collection(collection, out_dir, scale=30)

Total number of images: 3



Consider adjusting `region`, `scale` and/or `dtype` to reduce the LC08_019049_20240211.tif download size (raw: 4.41 GB).


LC08_019049_20240211.tif: |                                           | 0.00/4.41G (raw) [  0.0%] in 00:00 (et…

Consider adjusting `region`, `scale` and/or `dtype` to reduce the LC08_019050_20240211.tif download size (raw: 4.41 GB).


LC08_019050_20240211.tif: |                                           | 0.00/4.41G (raw) [  0.0%] in 00:00 (et…

KeyboardInterrupt: 

In [6]:
# geemap.download_ee_image_tiles(
#     composite, features, out_dir, prefix="landsat_", crs="EPSG:3857", scale=30
# )

ValueError: features must be an ee.FeatureCollection.

In [8]:
# Define your study area in Guatemala (example coordinates)
# Replace these coordinates with your actual study area
guatemala_aoi = ee.Geometry.Polygon([
        [-90.1517107831, 15.5726233568],
        [-89.1726311637, 15.5726233568],
        [-89.1726311637, 16.29215598],
        [-90.1517107831, 16.29215598],
        [-90.1517107831, 15.5726233568]
])

# Set your output folder
output_folder = "landsat_images"

# Download images
download_landsat_images(
    aoi=guatemala_aoi,
    output_folder=output_folder,
    start_date='2023-01-01',
    end_date='2023-03-01',
    cloud_threshold=20,  # Adjust based on your needs
    max_images=3
)

Generating URL ...
Please wait ...
An error occurred while downloading.
Error downloading image: Expecting value: line 1 column 1 (char 0)
Generating URL ...
Please wait ...
An error occurred while downloading.
Error downloading image: Expecting value: line 1 column 1 (char 0)
Generating URL ...
Please wait ...
An error occurred while downloading.
Error downloading image: Expecting value: line 1 column 1 (char 0)
Generating URL ...



KeyboardInterrupt



In [5]:
Map = geemap.Map()
Map.addLayer(guatemala_aoi, {}, "Fire AOI")
Map.centerObject(guatemala_aoi, 7)
Map

Map(center=[15.93272670286817, -89.66217097339984], controls=(WidgetControl(options=['position', 'transparent_…

In [20]:
def get_least_cloudy_landsat(
    region,
    start_date,
    end_date,
    cloud_threshold=20
):
    """
    Get the least cloudy Landsat 8 image for a given time range
    
    Parameters:
    region (ee.Geometry): Region of interest
    start_date (str): Start date in 'YYYY-MM-DD' format
    end_date (str): End date in 'YYYY-MM-DD' format
    cloud_threshold (float): Maximum cloud cover percentage
    
    Returns:
    ee.Image: Least cloudy image
    """
    # Import Landsat 8 Collection 2 Level 2
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    
    # Filter collection
    filtered = collection.filterBounds(region)\
                        .filterDate(start_date, end_date)\
                        .filter(ee.Filter.lt('CLOUD_COVER', cloud_threshold))\
                        .sort('CLOUD_COVER')  # Sort by cloud cover
    
    # Get the least cloudy image
    image = filtered.first()
    
    # Select RGB bands and scale them
    # For Landsat 8 C2 L2, RGB = B4, B3, B2
    rgb_image = image.select(['SR_B4', 'SR_B3', 'SR_B2'])\
                     .multiply(0.0000275)\
                     .add(-0.2)  # Apply scaling factors
    
    return rgb_image, image.get('CLOUD_COVER')

def create_fishnet_and_download(
    image,
    region,
    out_dir,
    date_str,
    rows=2,
    cols=2,
    prefix="landsat8_rgb_",
    crs="EPSG:3857",
    scale=30
):
    """
    Creates a fishnet grid and downloads RGB image tiles
    """
    # Create fishnet grid
    features = geemap.fishnet(region, rows=rows, cols=cols)
    
    # Download tiles
    geemap.download_ee_image_tiles(
        image=image,
        features=features,
        out_dir=out_dir,
        prefix=f"{prefix}{date_str}_",
        crs=crs,
        scale=scale
    )

In [25]:
import ee
import geemap
import os
from datetime import datetime

# Initialize Earth Engine with your project
ee.Initialize(project="ee-samettenborough")

def get_least_cloudy_landsat(region, start_date, end_date, cloud_threshold=20):
    """
    Get the least cloudy Landsat 8 image for a given time range
    """
    # Import Landsat 8 Collection 2 Level 2
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    
    # Filter collection
    filtered = collection.filterBounds(region)\
                        .filterDate(start_date, end_date)\
                        .filter(ee.Filter.lt('CLOUD_COVER', cloud_threshold))\
                        .sort('CLOUD_COVER')
    
    # Select RGB bands and scale them
    rgb_image = image.select(['SR_B4', 'SR_B3', 'SR_B2'])\
                     .multiply(0.0000275)\
                     .add(-0.2)
    
    return rgb_image, image.get('CLOUD_COVER')

# Create a map
Map = geemap.Map()

# Define your parameters
start_date = '2023-01-01'
end_date = '2023-03-01'
cloud_threshold = 20

# Define region (Guatemala example - modify these coordinates for your area)
region = ee.Geometry.BBox(-90.1517107831, 15.5726233568, -89.1726311637, 16.29215598)

# Get the least cloudy image
rgb_image, cloud_cover = get_least_cloudy_landsat(
    region=region,
    start_date=start_date,
    end_date=end_date,
    cloud_threshold=cloud_threshold
)

# Add the region boundary to the map
Map.addLayer(region, {'color': 'red'}, 'Study Area')

# Add the RGB image to the map
vis_params = {
    'min': 0,
    'max': 0.3,
    'gamma': 1.4
}
Map.addLayer(rgb_image, vis_params, 'Landsat 8 RGB')

# Center the map on the region
Map.centerObject(region, 10)

In [26]:
Map

Map(center=[15.932358717055706, -89.66217097339984], controls=(WidgetControl(options=['position', 'transparent…

In [34]:
def get_unique_path_rows(region, start_date, end_date):
    """
    Get unique WRS path/row combinations for a region
    """
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        .filterBounds(region)\
        .filterDate(start_date, end_date)
    
    # Get unique path-row combinations
    unique_pr = collection.aggregate_array('WRS_PATH')\
        .zip(collection.aggregate_array('WRS_ROW'))\
        .distinct()
    
    return unique_pr.getInfo()

def get_least_cloudy_for_path_row(region, start_date, end_date, path, row, cloud_threshold=20):
    """
    Get least cloudy image for specific path/row
    """
    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        .filterBounds(region)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.eq('WRS_PATH', path))\
        .filter(ee.Filter.eq('WRS_ROW', row))\
        .filter(ee.Filter.lt('CLOUD_COVER', cloud_threshold))\
        .sort('CLOUD_COVER')
    
    # Get the least cloudy image
    image = collection.first()
    
    if image is None:
        return None, None
    
    # Select RGB bands and scale them
    rgb_image = image.select(['SR_B4', 'SR_B3', 'SR_B2'])\
        .multiply(0.0000275)\
        .add(-0.2)
    
    return rgb_image, image.get('CLOUD_COVER')

# Create a map
Map = geemap.Map()

# Define your parameters
start_date = '2023-01-01'
end_date = '2023-03-01'
cloud_threshold = 20

# Define region (Guatemala example - modify these coordinates for your area)
region = ee.Geometry.BBox(-90.1517107831, 15.5726233568, -89.1726311637, 16.29215598)

# Get unique path/row combinations
unique_path_rows = get_unique_path_rows(region, start_date, end_date)
print(f"Found {len(unique_path_rows)} unique path/row combinations:")
for path, row in unique_path_rows:
    print(f"Path: {path}, Row: {row}")

# Process each path/row combination
vis_params = {
    'min': 0,
    'max': 0.3,
    'gamma': 1.4
}

# Add the region boundary to the map
Map.addLayer(region, {'color': 'red'}, 'Study Area')

# Set up the output directory
out_dir = os.path.expanduser("~/Downloads/landsat_rgb")
os.makedirs(out_dir, exist_ok=True)

# Process each path/row combination
for path, row in unique_path_rows:
    rgb_image, cloud_cover = get_least_cloudy_for_path_row(
        region=region,
        start_date=start_date,
        end_date=end_date,
        path=path,
        row=row,
        cloud_threshold=cloud_threshold
    )
    
    if rgb_image is not None:
        # Add the RGB image to the map
        layer_name = f'Landsat 8 RGB (Path {path}, Row {row})'
        Map.addLayer(rgb_image, vis_params, layer_name)
        
        # Create fishnet grid for this path/row
        features = geemap.fishnet(region, rows=2, cols=2)
        grid_name = f'Download Grids (Path {path}, Row {row})'
        Map.addLayer(features, {'color': 'yellow'}, grid_name)
        
        # Download the tiles
        date_str = datetime.now().strftime('%Y%m%d')
        prefix = f"landsat8_rgb_p{path}r{row}_{date_str}_"
        
        geemap.download_ee_image_tiles(
            image=rgb_image,
            features=features,
            out_dir=out_dir,
            prefix=prefix,
            crs="EPSG:3857",
            scale=30
        )
        
        print(f"Processed Path {path}, Row {row} - Cloud cover: {cloud_cover.getInfo():.2f}%")
    else:
        print(f"No suitable images found for Path {path}, Row {row}")

# Center the map on the region
Map.centerObject(region, 10)

# Display the map
Map

Found 2 unique path/row combinations:
Path: 19, Row: 49
Path: 20, Row: 49


landsat8_rgb_p19r49_20250117_1.tif: |                                 | 0.00/60.6M (raw) [  0.0%] in 00:00 (et…

landsat8_rgb_p19r49_20250117_2.tif: |                                 | 0.00/60.5M (raw) [  0.0%] in 00:00 (et…

landsat8_rgb_p19r49_20250117_3.tif: |                                 | 0.00/60.6M (raw) [  0.0%] in 00:00 (et…

landsat8_rgb_p19r49_20250117_4.tif: |                                 | 0.00/60.7M (raw) [  0.0%] in 00:00 (et…

...

landsat8_rgb_p20r49_20250117_7.tif: |                                 | 0.00/60.8M (raw) [  0.0%] in 00:00 (et…

landsat8_rgb_p20r49_20250117_8.tif: |                                 | 0.00/60.7M (raw) [  0.0%] in 00:00 (et…

landsat8_rgb_p20r49_20250117_9.tif: |                                 | 0.00/60.8M (raw) [  0.0%] in 00:00 (et…

Downloaded 9 tiles in 33.98576760292053 seconds.
Processed Path 20, Row 49 - Cloud cover: 6.36%


Map(center=[15.932358717055706, -89.66217097339984], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
def create_fishnet_and_download(
    image,
    region,
    out_dir,
    rows=2,
    cols=2,
    prefix="landsat_",
    crs="EPSG:3857",
    scale=30
):
    """
    Creates a fishnet grid and downloads image tiles without visualization
    
    Parameters:
    image (ee.Image): Earth Engine image to download
    region (ee.Geometry): Region to download
    out_dir (str): Output directory path
    rows (int): Number of rows in the grid
    cols (int): Number of columns in the grid
    prefix (str): Prefix for output filenames
    crs (str): Coordinate reference system
    scale (int): Resolution in meters
    """
    # Create fishnet grid
    features = geemap.fishnet(region, rows=rows, cols=cols)
    
    # Download tiles
    geemap.download_ee_image_tiles(
        image=image,
        features=features,
        out_dir=out_dir,
        prefix=prefix,
        crs=crs,
        scale=scale
    )
    
# Initialize Earth Engine with your project
ee.Initialize(project="ee-samettenborough")

# Set output directory
out_dir = os.path.expanduser("~/Downloads/landsat_tiles")
os.makedirs(out_dir, exist_ok=True)

# Load the image
image = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")

# Define region (Guatemala example)
region = ee.Geometry.BBox(-90.1517107831, 15.5726233568, -89.1726311637, 16.29215598)

# Create fishnet and download tiles
create_fishnet_and_download(
    image=image,
    region=region,
    out_dir=out_dir,
    rows=2,
    cols=2,
    prefix="landsat_",
    scale=30
)

print(f"Download complete. Files saved to: {out_dir}")